SyntaxError: invalid syntax (3805922518.py, line 1)

In [1]:
import torch
import numpy as np
import os
import sys

src_path = os.path.abspath(os.path.join("src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(sys.path)

['/storage/ice1/7/8/abhalerao9/tycho/TimeSeriesModeling/TSFMProject/src', '/home/hice1/abhalerao9/scratch/conda/envs/timesfm/lib/python311.zip', '/home/hice1/abhalerao9/scratch/conda/envs/timesfm/lib/python3.11', '/home/hice1/abhalerao9/scratch/conda/envs/timesfm/lib/python3.11/lib-dynload', '', '/home/hice1/abhalerao9/scratch/conda/envs/timesfm/lib/python3.11/site-packages', '/storage/ice1/7/8/abhalerao9/tycho/TimeSeriesModeling/TSFMProject/src/tsfmproject/models/timesfm']


In [2]:
from tsfmproject.model import ChronosModel
from tsfmproject.dataset import ChronosDataset
from tsfmproject.visualization import ForecastVisualization

/home/hice1/abhalerao9/scratch/conda/envs/timesfm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TimesFM v1.2.0. See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM.


In [3]:
# Get the base path from sys.path[0]
base_path = sys.path[0]

# Relative file location
relative_file_location = '../data/dataset/ETTh1.csv'

# Combine the base path with the relative file location
file_location = os.path.join(base_path, relative_file_location)


Chdataset = ChronosDataset(name="ett", path=file_location, datetime_col='date', freq='H')
Chdataset.preprocess()
data = Chdataset.dataset
data_loc = os.path.join(sys.path[0],'./tsfmproject/models/chronosforecasting/data/us_data.arrow')
offset = -60
# slice data in the last offset rows
data_train = data.iloc[:offset]
data_test = data.iloc[offset:]
start_date = str(data_train.index[0])
# make a list of the columns except 'ds' columns and make a list of all the time series in those columns
column_list = data_train.columns.tolist()
time_series_list = [np.array(data_train[column].values) for column in column_list]

In [4]:
print(time_series_list)

[array([ 5.82700014,  5.69299984,  5.15700006, ..., 15.73999977,
       13.73099995,  9.04199982]), array([2.00900006, 2.07599998, 1.74100006, ..., 5.76000023, 4.95699978,
       5.42500019]), array([ 1.59899998,  1.49199998,  1.27900004, ..., 10.16300011,
        8.24400043,  3.30500007]), array([0.46200001, 0.426     , 0.35499999, ..., 2.91400003, 2.0250001 ,
       3.05599999]), array([4.20300007, 4.1420002 , 3.77699995, ..., 5.39099979, 5.4829998 ,
       5.39099979]), array([1.34000003, 1.37100005, 1.21800005, ..., 1.67499995, 1.73599994,
       1.76699996]), array([30.53100014, 27.78700066, 27.78700066, ...,  6.89400005,
        7.45699978,  7.45699978])]


In [17]:
Chdataset.convert_to_arrow(data_loc, time_series=time_series_list, start_date=start_date)

In [6]:
repo = "amazon/chronos-t5-small"
ch = ChronosModel(config=None, repo=repo)

In [ ]:
ch.finetune(training_data_paths=[data_loc], probability_list=[1])

The speedups for torchdynamo mostly come wih GPU Ampere or higher and which is not detected here.
max_steps is given, it will override any value given in num_train_epochs
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [7]:
results = {}
model_dir = None
# latest_run_dir = ch.get_latest_run_dir()
# model_dir = os.path.join(latest_run_dir, "checkpoint-final")
# model_type = "seq2seq"
metrics = ['RMSE', 'MAPE']
model = ch.load_model()
ch.result_logger.info(f"Model loaded from {model_dir}")

In [8]:
eval_results, trues, preds, histories = ch.evaluate(data_train, data_test, metrics)

KeyboardInterrupt: 